In [1]:
import networkx as nx
import pickle
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore') 

In [2]:
def compute_risk_fullpath(pt_file: str, pickle_file: str) -> float:
    """
    Load tensor output from path prediction model
    """
    y_pred = torch.load(pt_file).numpy()

    """
    Load graph from .pickle file
    """
    with open(pickle_file, "rb") as f:
        G = pickle.load(f)

    """
    Sanity check for graph order
    """
    nodes = sorted(G.nodes())
    num_nodes = len(nodes)
    if y_pred.shape != (num_nodes, num_nodes):
        raise ValueError("Mismatch between adjacency matrix size and graph size!")

    """
    Compute reverse pagerank
    """
    G = G.reverse()
    rpr = nx.pagerank(G)

    """
    Compute risk(G)
    """
    risk = 0.0
    for i, node in enumerate(nodes):
        avg_prob = np.mean(y_pred[i])  
        risk += avg_prob * rpr.get(node, 0)
        
    return risk*100


"""
Compute risk for sample graph
"""

Y = "Dataset/DemoRisk/Y_pred.pt"
X = "Dataset/DemoRisk/graph_0I9E8nlQ.pickle"
risk = compute_risk_fullpath(Y, X)
print(f"[+] Risk for full path: {risk:.2f} %")

[+] Risk for full path: 7.33 %


In [3]:
def compute_weighted_risk(adj_pt_file: str, graph_pickle: str, weights_pt_file: str) -> float:
    
    """
    Compute the weighted risk as:
    sum(weight[i] * compute_risk_fullpath[i])
    """
    weights = torch.load(weights_pt_file).numpy()

    """
    Load graph from .pickle file
    """
    with open(graph_pickle, "rb") as f:
        G = pickle.load(f)

    """
    Sanity check for graph order
    """
    nodes = sorted(G.nodes())
    num_nodes = len(nodes)
    if weights.shape != (num_nodes,):
        raise ValueError("Mismatch between weight tensor size and number of nodes in graph!")

    """
    Compute per-node risk using compute_risk_fullpath
    """
    risk_per_node = compute_risk_fullpath(adj_pt_file, graph_pickle)

    """
    Compute final weighted risk sum
    """
    weighted_risk = np.sum(weights * risk_per_node)
    return (weighted_risk/num_nodes)


"""
Compute weighted risk for sample graph
"""

Y = "Dataset/DemoRisk/Y_pred.pt"  
X = "Dataset/DemoRisk/graph_0I9E8nlQ.pickle"
W = "Dataset/DemoRisk/f_start.pt" 

weighted_risk = compute_weighted_risk(Y, X, W)
print(f"[+] Risk for start node: {weighted_risk:.2f} %")

[+] Risk for start node: 3.55 %


In [4]:
def compute_weighted_risk(adj_pt_file: str, graph_pickle: str, weights_pt_file: str) -> float:
    
    """
    Compute the weighted risk as:
    sum(weight[i] * compute_risk_fullpath[i])
    """
    weights = torch.load(weights_pt_file).numpy()

    """
    Load graph from .pickle file
    """
    with open(graph_pickle, "rb") as f:
        G = pickle.load(f)

    """
    Sanity check for graph order
    """
    nodes = sorted(G.nodes())
    num_nodes = len(nodes)
    if weights.shape != (num_nodes,):
        raise ValueError("Mismatch between weight tensor size and number of nodes in graph!")

    """
    Compute per-node risk using compute_risk_fullpath
    """
    risk_per_node = compute_risk_fullpath(adj_pt_file, graph_pickle)

    """
    Compute final weighted risk sum
    """
    weighted_risk = np.sum(weights * risk_per_node)
    return (weighted_risk/num_nodes)


"""
Compute weighted risk for sample graph
"""

Y = "Dataset/DemoRisk/Y_pred.pt"  
X = "Dataset/DemoRisk/graph_0I9E8nlQ.pickle"
W = "Dataset/DemoRisk/f_end.pt" 

weighted_risk = compute_weighted_risk(Y, X, W)
print(f"[+] Risk for end node: {weighted_risk:.2f} %")

[+] Risk for end node: 3.66 %
